In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [ ]:
import json
import numpy as np
import pandas as pd         
import ctypes

In [ ]:
hospitals = pd.read_csv('https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/GA_Hospitals.csv')
hospitals = hospitals.fillna('')
hospitals['Med_center'] = hospitals['Med_center'] + " " + hospitals['Unnamed: 2']
hospitals = hospitals.drop('Unnamed: 2', axis=1)

baseurl = 'https://freida.ama-assn.org/institution/'
URLS = []
for index, i in hospitals.iterrows():
  URLS.append(str(baseurl + str(i['H.No.'])))
hospitals['URLS'] = URLS

hospitals.head()

In [ ]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument("enable-automation")
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-browser-side-navigation")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

def pagesource(url, driver):
    driver = driver
    driver.get(url)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source)
    #driver.close()
    return soup



In [ ]:
# d = driversetup()
# relationship = []
# for index, i in hospitals.iterrows():
#   url = i['URLS']
#   s = pagesource(url, d)
#   med_school = list(s.findAll('small', class_="bold"))
#   print(i.Med_center, index, relationship, len(med_school))
#   for i in range(len(med_school)):
#     a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    
#     for i in range(len(a)):
#       rel = a[i].findAll('div', class_='')[0].text
#       if rel not in relationship:
#         print(med_school[i].text, rel)
#         relationship.append(rel)
# relationship

In [ ]:
mc = []
grad = []
mcl = []
add = []
d = driversetup()
for index, i in hospitals.iterrows():
  url = i['URLS']
  #url = 'https://freida.ama-assn.org/institution/120119'
  s = pagesource(url, d)
  med_school = list(s.findAll('small', class_="bold"))
  address = list(s.findAll('div', class_="details__address ng-star-inserted")) 
  if len(med_school) == 0:
    mc.append("")
    mcl.append("")
    grad.append("")
  elif len(med_school) == 1:
    a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    name = a[0].findAll('small', class_='bold')[0].text
    relationship = a[0].findAll('div', class_='')[0].text
    print(name, relationship)
    if 'Limited' in relationship:
      mcl.append(name)
      mc.append("")
      grad.append("")
    elif 'Major' in relationship:
      mc.append(name)
      mcl.append("")
      grad.append("")
    elif 'Graduate' in relationship:
      mc.append("")
      mcl.append("")
      grad.append(name)
    else:
      print("NEW RELATIONSHIP FOUND : ", name)
      break
  elif len(med_school) > 1:   
      major = []
      limited = []
      graduate = []
      a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
      for i in range(len(a)):
        name = a[i].findAll('small', class_='bold')[0].text
        relationship = a[i].findAll('div', class_='')[0].text
        print(i, name, relationship)
        if 'Limited' in relationship:
          limited.append(name)
        elif 'Graduate' in relationship:
          graduate.append(name)
        elif 'Major' in relationship:
          major.append(name)
        else:
          print("New relationship found")
          break 
      if major == []:
        major = ''
      if limited == []:
        limited = ''
      if grad == []:
        grad = ''
      mc.append(major)
      mcl.append(limited)
      grad.append(graduate)
  if len(address) != 0:
    address[0].br.string = ', '
    add.append(address[0].text)
  else:
    add.append("")
  print(index, 'mc', len(mc), mc[index], 'mcl', len(mcl), mcl[index], 'grad', len(grad), grad[index], 'add', len(add), add[index])
  if len(mc) > (index+1):
    break



In [ ]:
# print(len(mc),len(mcl),len(grad),len(add))
# hospitals['Affiliation_Major'] = mc
# hospitals['Affiliation_Limited'] = mcl
# hospitals['Affiliation_Graduate'] = grad
# hospitals['Medical_Center_Address'] = add

# hospitals.iloc[[40]]

hospitals.to_csv('GA_crosswalk.csv')